In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cosinesim-ft-talent-v1/COSINESIM_FINETUNE_DATASET_16k_final.csv
/kaggle/input/ml-talent-hack-parsedtrain/output.csv


## **paraphrase-multilingual-mpnet-base-v2 (Multilingual XLM-roBERTa) FINE-TUNING**

# Here, we assume our dataset for fine-tuning is fully ready (text corpuses are preproccessed, cosine similarity is calcluated):

In [2]:
df = pd.read_csv('/kaggle/input/cosinesim-ft-talent-v1/COSINESIM_FINETUNE_DATASET_16k_final.csv')
df.sample(3)

,CONCATED_x,CONCATED_y,COSINE_SIM
369,ЗП: 40000 руб.. Ищет работу на должность:: Инж...,title: Senior DevOps Engineer. salary: з/п не ...,0.517165
833,ЗП: 70000 руб.. Ищет работу на должность:: Про...,title: Бэкенд-разработчик. salary: от 120000 д...,0.521445
13180,ЗП: 55000 руб.. Ищет работу на должность:: Инж...,title: Go разработчик. salary: з/п не указана....,0.536804


## Let's import and initialize everything we need:

In [6]:
# !pip install sentence-transformers

In [38]:
from transformers import AutoTokenizer, AutoModel
from sentence_transformers.losses import ContrastiveLoss
from sentence_transformers import SentenceTransformer
import torch
import torch.nn as nn

model = AutoModel.from_pretrained('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

#for clean PyTorch this can be used:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] 
    #as it contains all embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(toke_embeddings.size()).float()
    #att mask size -> token embs size
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    #paddings to zero
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    #normalization excluding zero
    
    return sum_embeddings / sum_mask #returning avg

#for simplicity let's use one already incl. pooling
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.13k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Case without splitting texts into corpuses as we will fine-tune on the full corpus data embeddings' similarity with dropping several columns (check README for reference):

In [10]:
from sentence_transformers import InputExample

val_data = pd.read_csv('/kaggle/input/ml-talent-hack-parsedtrain/output.csv')
cols = list(np.array(val_data.columns))
print(cols)

['Vacancy UUID', 'Vacancy Name', 'Keywords', 'Description', 'Comment', 'Resume UUID', 'First Name', 'Last Name', 'Birth Date', 'Country', 'City', 'About', 'Key Skills', 'Starts', 'Ends', 'Employer', 'Experience City', 'Position', 'Experience Description', 'Year', 'Organization', 'Faculty', 'Specialty', 'Result', 'Education Type', 'Education Level', 'Target']


In [20]:
cols_drop1 = cols[0:9] + cols[13:16] + [cols[19]] + [cols[-1]]
resumes_val = val_data.drop(columns=cols_drop1)
vac_val = val_data.iloc[:, 1:5]
labels_val = np.array(pd.get_dummies(val_data.iloc[:, -1]).astype(int).drop(columns='failed'))

In [16]:
resumes_val_text = []
for i in range(len(resumes_val)):
    curr_str = ""
    for j in range(len(resumes_val.columns)):
        if not pd.isna(resumes_val.iloc[i, j]):
            curr_str += str(resumes_val.iloc[i, j]) + ". "
    resumes_val_text.append(curr_str)
vac_val_text = []
for i in range(len(vac_val)):
    curr_str = ""
    for j in range(len(vac_val.columns)):
        if not pd.isna(vac_val.iloc[i, j]):
            curr_str += str(vac_val.iloc[i, j]) + ". "
    vac_val_text.append(curr_str)

In [21]:
len(vac_val_text), len(resumes_val_text), len(labels_val)

(656, 656, 656)

In [148]:
from torch.utils.data import DataLoader

val_dataset = [InputExample(texts=[resumes_val_text[i], vac_val_text[i]], label=labels_val[i]) for i in range(len(vac_val_text))]
train_dataset = [InputExample(texts=[df.iloc[i, 0], df.iloc[i, 1]], label=df.iloc[i, 2]) for i in range(len(df))]

def collate_fn(batch):
    texts = [example.texts for example in batch]
    labels = torch.tensor([example.label for example in batch])
    return texts, labels

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=32, collate_fn=collate_fn)
val_dataloader = DataLoader(val_dataset, shuffle=False, batch_size=16, collate_fn=collate_fn)

# Testing metrics first:
### With threshold = 0.65

In [76]:
from tqdm import tqdm
from IPython.display import clear_output
eval_preds, eval_labels = [], []

model.eval()
with torch.no_grad():
    for batch in tqdm(val_dataloader):
        clear_output()
        texts, labels = batch
        emb1 = model.encode([x[0] for x in texts], convert_to_tensor=True, normalize_embeddings=False)
        emb2 = model.encode([x[1] for x in texts], convert_to_tensor=True, normalize_embeddings=False)
        cos_sim = nn.functional.cosine_similarity(emb1, emb2)
        preds = (cos_sim > 0.65).int()
        eval_preds.append(preds.cpu().tolist())
        eval_labels.append(labels.cpu().tolist())

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 41/41 [00:06<00:00,  6.42it/s]


In [77]:
eval_preds = np.array(eval_preds).reshape(-1, 1)
eval_labels = np.array(eval_labels).reshape(-1, 1)

In [78]:
from sklearn.metrics import classification_report

print(classification_report(eval_labels, eval_preds)) #DEFAULT

              precision    recall  f1-score   support

           0       0.70      0.97      0.82       463
           1       0.19      0.02      0.03       193

    accuracy                           0.69       656
   macro avg       0.45      0.49      0.42       656
weighted avg       0.55      0.69      0.58       656



In [162]:
from tqdm import tqdm
from IPython.display import clear_output
eval_preds, eval_labels = [], []

model.eval()
with torch.no_grad():
    for batch in tqdm(val_dataloader):
        clear_output()
        texts, labels = batch
        emb1 = model.encode([x[0] for x in texts], convert_to_tensor=True, normalize_embeddings=False)
        emb2 = model.encode([x[1] for x in texts], convert_to_tensor=True, normalize_embeddings=False)
        cos_sim = nn.functional.cosine_similarity(emb1, emb2)
        preds = (cos_sim > 0.65).int()
        eval_preds.append(preds.cpu().tolist())
        eval_labels.append(labels.cpu().tolist())

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 41/41 [00:06<00:00,  6.63it/s]


In [163]:
eval_preds = np.array(eval_preds).reshape(-1, 1)
eval_labels = np.array(eval_labels).reshape(-1, 1)

In [164]:
print(classification_report(eval_labels, eval_preds)) 

              precision    recall  f1-score   support

           0       0.70      0.97      0.82       463
           1       0.19      0.02      0.03       193

    accuracy                           0.69       656
   macro avg       0.45      0.49      0.42       656
weighted avg       0.55      0.69      0.58       656



### Acceptable, but we want more.

In [132]:
tqdm_bar = None
tqdm = None

# TODO: CUSTOM DATASET CLASS

In [ ]:
from transformers import AdamW
from tqdm import tqdm
tqdm_notebook = tqdm
def nop(it, *a, **k):
    return it
tqdm_notebook.tqdm = nop

class ContrastiveLoss_v1(nn.Module):
    def __init__(self, margin=2.0):
        super(ContrastiveLoss_v1, self).__init__()
        self.margin = margin

    def forward(self, y1, y2, labels):
        # euclid
        euc_dist = nn.functional.pairwise_distance(y1, y2)
        
        # calc losses
        losses = (1 - labels) * torch.pow(euc_dist, 2) + \
                 labels * torch.pow(torch.clamp(self.margin - euc_dist, min=0.0), 2)
                 
        loss = torch.mean(losses)
        return loss


model.to('cuda')

loss_func = ContrastiveLoss_v1()
optimizer = AdamW(model.parameters(), lr=2e-5)

NUM_EPOCHS = 2

for epoch in tqdm(range(NUM_EPOCHS)):
    print(f"EPOCH {epoch+1}")
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader, desc=f"Epoch {epoch+1}", unit="batch")):
        optimizer.zero_grad()
        
        texts, labels = batch
        sentences_cv = [text[0] for text in texts]
        sentences_vac = [text[1] for text in texts]
        
        embs1 = model.encode(sentences_cv, convert_to_tensor=True, normalize_embeddings=False).requires_grad_()
        embs2 = model.encode(sentences_vac, convert_to_tensor=True, normalize_embeddings=False).requires_grad_()
        loss = loss_func(embs1.to('cuda'), embs2.to('cuda'), labels.to('cuda'))
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
        if step % 100 == 1:
            print(f"Step {step}: Loss - {total_loss/step}")
        
    avg_loss = total_loss / len(train_dataloader)
    print(f"Training loss: {avg_loss:.4f}")
    
    ###val below
    
    model.eval()
    eval_preds, eval_preds1, eval_preds2 = [], [], []
    with torch.no_grad():
        val_loss = 0
        for step, batch in enumerate(val_dataloader):
            texts, labels = batch
            sentences_cv = [text[0] for text in texts]
            sentences_vac = [text[1] for text in texts]

            embs1 = model.encode(sentences_cv, convert_to_tensor=True, normalize_embeddings=False)
            embs2 = model.encode(sentences_vac, convert_to_tensor=True, normalize_embeddings=False)
            
            loss = loss_func(embs1.to('cuda'), embs2.to('cuda'), labels.to('cuda'))
            val_loss += loss.item()
            
            cos_sim = nn.functional.cosine_similarity(embs1, embs2)
            preds = (cos_sim > 0.4).int()
            preds1 = (cos_sim > 0.5).int()
            preds2 = (cos_sim > 0.6).int()
            eval_preds.append(preds.cpu().tolist())
            eval_preds1.append(preds1.cpu().tolist())
            eval_preds2.append(preds2.cpu().tolist())
            
    eval_preds = np.array(eval_preds).reshape(-1, 1)
    eval_preds1 = np.array(eval_preds1).reshape(-1, 1)
    eval_preds2 = np.array(eval_preds2).reshape(-1, 1)
    
            
    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Validation loss: {avg_val_loss:.4f}\n")
    print("THRESHOLD 0.4:\n", classification_report(eval_labels, eval_preds))
    print("THRESHOLD 0.5:\n", classification_report(eval_labels, eval_preds1))
    print("THRESHOLD 0.6:\n", classification_report(eval_labels, eval_preds2))    

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/2 [00:00<?, ?it/s]

EPOCH 1



Epoch 1:   0%|          | 0/497 [00:00<?, ?batch/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...


Epoch 1:  79%|███████▉  | 395/497 [02:26<00:36,  2.77batch/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Epoch 1:  80%|███████▉  | 396/497 [02:27<00:36,  2.80batch/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [156]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [96]:
losses.ContrastiveLoss()

pytorch_metric_learning.losses.contrastive_loss.ContrastiveLoss

In [86]:
2e-5 > 0

True